# Control

I'm using this notebook to:

1. Explore what it takes to add a new control dataset.
2. Make notes that we can use for discussion and knowledge-share.

My plan, for now, is to copy into this notebook the bare minimum lines needed to run a training/evaluation step using a ControlTask.

Every time I hit a road block, I'll make note of the issue, try my best to get around it (not necessarily in the correct way), and rinse/repeat.

If you're scanning for places where I have questions, look for formatting ***? like this ?***.

In [1]:
# If you edit code while this notebook is running,
# you might need to reload the module for the changes
# to take effect. You could also resolve the issue
# by restarting the kernel. But reloading the module
# is quicker and less destructive. I'm adding the 
# reload at the top so you can just run this cell
# after making any breaking edits to the code.
import importlib
import gato.tasks.control_task
importlib.reload(gato.tasks.control_task)

import minari
import gymnasium as gym
from gato.tasks.control_task import ControlTask

pygame 2.5.2 (SDL 2.28.2, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


## BabyAI Minari dataset

I grabbed a random dataset from the ones created with [the bot.py script in Santiago's baby-ai-dataset repo](https://github.com/snat-s/baby-ai-dataset/blob/master/scripts/bot.py#L64).

In [2]:
dataset_name = 'd4rl_halfcheetah-expert-v2'
dataset = minari.load_dataset(dataset_name)

In [3]:
dataset, dataset.spec.env_spec.id

(<minari.dataset.minari_dataset.MinariDataset at 0x7f40dc0f0130>,
 'HalfCheetah-v3')

In [4]:
env = gym.make(dataset.spec.env_spec)

/home/eihli/miniconda3/envs/neko/lib/python3.10/site-packages/gymnasium/envs/mujoco/mujoco_env.py:211: DeprecationWarning: WARN: This version of the mujoco environments depends on the mujoco-py bindings, which are no longer maintained and may stop working. Please upgrade to the v4 versions of the environments (which depend on the mujoco python bindings instead), unless you are trying to precisely replicate previous works).
  logger.deprecation(


# train.py

## Training arguments

I'm using [Namespace](https://docs.python.org/3/library/argparse.html#argparse.Namespace) to hack up an `args` object so that I don't have to go through the hassle of `parser = ArgumentParser; parser.add_argument(...); args = parser.parse(['foo', 'bar', 'baz', ...])`.

Browsing the code, the only `args` attribute that I see `ControlTask` access is `args.patch_size`, so that's the only one I'm bothering to add.

In [5]:
from argparse import Namespace

In [6]:
args = Namespace(patch_size=4)

In [7]:
context_len = 512
training_prompg_len_proportion=0.5
share_prompt_episodes=True
top_5_prompting=None

# ControlTask

`train.py` creates a `ControlTask`.

## Supported observation spaces

The first error I encounter is that the BabyAI environment has a Dict observation space.

***? What would it take to support a Dict (or any other) observation space ?***

Looking at how this is used, I see it conditionally:

- If it's a Box that has a shape of length 2 or 3
    - Adds image transforms
- Otherwise
    - Sets `obs_str` to `'continuous_obs'`, which eventually makes its way into `input_dict`, which eventually makes its way to `predict_control`, which doesn't get checked in `predict_control` but eventually makes its way to `tokenize_input_dicts`, and _that's_ where it gets checked.
        - It tokenizes the batch with the `continuous_obs_tokenizer`. 

(Is this a safe condition? Might there be non-image con)

### tokens_per_space

This is another place where the type of the observation space is checked.

Box: `space.shape[0]`
Discrete: `1`
Dict: ?

***? What should this? How is it used? ?***

`tokens_per_space` gets assigned to `action_tokens` [here](https://github.com/eihli/NEKO/blob/b66b48b88117307a442c43a7f4d8701706670144/gato/tasks/control_task.py#L74) and then used to calculate `tokens_per_timestamp` and is eventually used to create/manipulate the shape of the `input_dict` in [ControlTask.evaluate](https://github.com/eihli/NEKO/blob/b66b48b88117307a442c43a7f4d8701706670144/gato/tasks/control_task.py#L138).

### Hacking my way past errors.

I hard coded some arbitrary values to get past the assertions.

For example, I added an `isinstance(space, Dict): return 1` to `tokens_per_space` [here](https://github.com/eihli/NEKO/blob/b66b48b88117307a442c43a7f4d8701706670144/gato/tasks/control_task.py#L23).

I'm sure it's going to blow up due to a dict size mismatch or something. I just want to get to that point so I can see that error and maybe understand what the value _should_ be.

In [8]:
env.observation_space

Box(-inf, inf, (17,), float64)

In [9]:
control = ControlTask('playground_env_name', env, dataset, context_len, args)

# Back to train.py now that we have a ControlTask

Now we're going to need a lot more args.

In [10]:
from gato.policy.gato_policy import GatoPolicy

In [11]:
args.device = 'cpu'
args.embed_dim = 128
args.layers = 4
args.heads = 8
args.dropout = 0.1
args.mu = 100
args.M = 256
args.resid_mid_channels = 128
args.continuous_tokens = 1024
args.discrete_tokens = 1024
args.sequence_length = 1024
args.disable_patch_pos_encoding = False
args.disable_inner_pos_encoding = False
args.activation_fn = 'gelu'
args.pretrained_lm = None
args.flash = False
args.tokenizer_model_name = 'gpt2'
args.pad_seq = False

In [12]:
model = GatoPolicy(
    device=args.device,
    embed_dim=args.embed_dim,
    layers=args.layers,
    heads=args.heads,
    dropout=args.dropout,
    mu=args.mu,
    M=args.M,
    patch_size=args.patch_size,
    resid_mid_channels=args.resid_mid_channels,
    continuous_tokens=args.continuous_tokens,
    discrete_tokens=args.discrete_tokens,
    context_len=args.sequence_length,
    use_patch_pos_encoding=not args.disable_patch_pos_encoding,
    use_pos_encoding=not args.disable_inner_pos_encoding,
    activation_fn=args.activation_fn,
    pretrained_lm=args.pretrained_lm,
    flash=args.flash,
    tokenizer_model_name=args.tokenizer_model_name,
    pad_seq=args.pad_seq,
)

In [13]:
args.embed_dim = model.embed_dim

In [14]:
from accelerate import Accelerator, DistributedDataParallelKwargs

In [15]:
args.cpu = True
args.mixed_precision = 'no'
args.gradient_accumulation_steps = 1
args.eval_mode = 'deterministic'
args.save_dir = 'models'

In [16]:
ddp_kwargs = DistributedDataParallelKwargs(find_unused_parameters=True)

In [17]:
accelerator = Accelerator(
    cpu=args.cpu,
    mixed_precision=args.mixed_precision,
    split_batches=True,
    gradient_accumulation_steps=args.gradient_accumulation_steps,
    kwargs_handlers=[ddp_kwargs]
)

In [18]:
model = accelerator.prepare(model)

# Trainer

***? What if we wanted to skip the trainer? Could I just run `evaluate` on the ControlTask myself ?***

`trainer.train` runs `train_iteration`.

`train_iteration` runs `model.train()` (where `model` is an `nn.Module` ([docs](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)) and we don't extend its `train` method).

`GatoPolicy.forward` gets something called a "final_representation", which is the result of passing some token embeddings and a mask to the GPT2Model.

I'm getting a bit lost at this point. I'd love to have someone explain what's going on around this part of the code.

`forward` calls `tokenize_input_dicts(inputs)` [here](https://github.com/eihli/NEKO/blob/b66b48b88117307a442c43a7f4d8701706670144/gato/policy/gato_policy.py#L156). If your tracking `inputs`, which is probably an important variable to track`, then this line is probably important.

Continuing anyways...

We eventually call `predict_tokens` and return the `logits` (and conditionally the `loss`). [predict_token](https://github.com/eihli/NEKO/blob/b9facb61e7d48bf5f9fef9f4ec73b85b531e4aaf/gato/policy/gato_policy.py#L123) is an `nn.Linear`.

In [19]:
import torch
from gato.training.schedulers import get_linear_warmup_cosine_decay_scheduler

In [20]:
args.learning_rate = 1e-4
args.beta_1 = 0.9
args.beta_2 = 0.95
args.adam_eps = 1e-8
args.weight_decay = 0.1
args.warmup_steps = 10
args.training_steps = 20
args.init_lr = 1e-7
args.min_factor = 10.0
args.disable_cosine_decay = False

In [21]:
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=args.learning_rate,
    betas=(args.beta_1, args.beta_2),
    eps=args.adam_eps,
    weight_decay=args.weight_decay,
)

# Setup scheduler
scheduler = get_linear_warmup_cosine_decay_scheduler(
    optimizer,
    args.warmup_steps,
    args.training_steps,
    base_lr=args.learning_rate,
    init_lr=args.init_lr,
    min_lr=args.learning_rate / args.min_factor,
    cosine_decay=not args.disable_cosine_decay
)

In [22]:
optimizer, scheduler = accelerator.prepare(optimizer, scheduler)

In [23]:
from gato.training.trainer import Trainer

In [24]:
from datetime import datetime

In [25]:
tasks = [control]
exp_name = f'neko-lab-{datetime.now().isoformat()}'

In [26]:
args.log_eval_freq = 10  # must be less than half of training_steps
args.save_model = False
args.save_mode = 'last'
args.text_prop = 0.5
args.batch_size = 512
args.prompt_ep_proportion = 0.25
args.prompt_len_proportion = 0.5

In [27]:
trainer = Trainer(
    model,
    optimizer,
    accelerator,
    scheduler,
    tasks,
    exp_name,
    args
)

In [ ]:
trainer.train()

# Pause...

I just realized it would probably benefit me to explore what an existing _working_ dataset looks like. I'm going to take a break from this notebook to go do that. Maybe I'll do it below. Maybe I'll do it in a new notebook.

In [ ]:
trainer

In [ ]:
args.training_steps, args.log_eval_freq, trainer.args.training_steps // trainer.args.log_eval_freq